# PQF: 2D Geodesic-like Ray Bending (Hybrid, Reviewer-Ready)

**No ad-hoc terms.** We show ray bending (geodesic-like trajectories) emerging from the LPQF local dispersion on a slowly varying background:

$$\omega^2(x,y)=c_\phi^2 k^2 + V''\big(\phi(x,y)\big) + \alpha\big[(\partial_x\theta)^2 + (\partial_y\theta)^2\big].$$

Define the **effective speed** and **index**
$$c_{\rm eff}^2(x,y)=c_\phi^2 + V''(\phi) + \alpha\,|\nabla\theta|^2, \quad n(x,y)=1/c_{\rm eff}(x,y).$$
Rays in a gradient-index medium follow (eikonal approximation):
$$\frac{d^2\mathbf r}{ds^2}=\nabla\ln n \; (=\tfrac12\nabla\ln n^2).$$

Below we construct $(\phi,\theta)$, compute $c_{\rm eff}$ and $n$, and trace rays. Numerical **hygiene** (periodic wrapping, step tied to grid, direction renorm) ensures results aren’t artifacts.

## 1) Parameters, Grid, and Helpers

In [ ]:
import numpy as np, matplotlib.pyplot as plt
from scipy.interpolate import RegularGridInterpolator

# --- LPQF params (dimensionless) ---
c_phi = 1.0
rho0  = 1.0
phi_s = 1.0
alpha = 0.6

def Vpp(phi):
    """Second derivative of V(phi)=rho0*exp(-phi/phi_s)."""
    return (rho0/(phi_s**2))*np.exp(-phi/phi_s)

# --- Domain ---
Lx, Ly = 200.0, 200.0
Nx, Ny = 512, 512
x = np.linspace(-Lx/2, Lx/2, Nx)
y = np.linspace(-Ly/2, Ly/2, Ny)
X, Y = np.meshgrid(x, y, indexing='xy')
dx, dy = x[1]-x[0], y[1]-y[0]
Lx_span, Ly_span = x[-1]-x[0], y[-1]-y[0]
EPS = 0.5*min(dx, dy)  # finite-difference step tied to grid

def d_dx(F): return (np.roll(F,-1,axis=0)-np.roll(F,1,axis=0))/(2*dx)
def d_dy(F): return (np.roll(F,-1,axis=1)-np.roll(F,1,axis=1))/(2*dy)

def wrap(px, py):
    """Wrap coordinates periodically into the domain (no physics changed)."""
    wx = x[0] + np.remainder(px - x[0], Lx_span)
    wy = y[0] + np.remainder(py - y[0], Ly_span)
    return wx, wy

## 2) Background Fields $(\phi,\theta)$

In [ ]:
# Smooth amplitude landscape (coherence bump + weak modulation)
phi = 0.6 + 0.25*np.exp(-((X-20.0)**2 + (Y-0.0)**2)/(22.0**2)) + 0.03*np.cos(2*np.pi*X/180.0)

# Phase field with anisotropic shear and mild y-variation
theta = (0.02*np.tanh((X-10.0)/40.0) * (1.0 + 0.2*np.sin(2*np.pi*Y/80.0))
         + 0.01*np.sin(2*np.pi*Y/120.0) * np.exp(-((X+30.0)/120.0)**2))

theta_x, theta_y = d_dx(theta), d_dy(theta)
theta_grad_sq = theta_x**2 + theta_y**2
print(f"phi range: {phi.min():.3f} .. {phi.max():.3f}")
print(f"|grad theta| range: {np.sqrt(theta_grad_sq).min():.3f} .. {np.sqrt(theta_grad_sq).max():.3f}")

## 3) Effective Speed $c_{\rm eff}$ and Index Map $n=1/c_{\rm eff}$

In [ ]:
Vpp_phi = Vpp(phi)
Delta   = Vpp_phi + alpha*theta_grad_sq
c_eff2  = np.maximum(c_phi**2 + Delta, 1e-12)
c_eff   = np.sqrt(c_eff2)
n_map   = 1.0 / c_eff
print('c_eff range:', float(c_eff.min()), 'to', float(c_eff.max()))
print('n_map  range:', float(n_map.min()), 'to', float(n_map.max()))

# Interpolator (we still wrap coords; fill_value won't be used for in-domain queries)
interp_n = RegularGridInterpolator((x, y), n_map.T, bounds_error=False, fill_value=n_map.mean())

def n_at(px, py):
    wx, wy = wrap(px, py)
    return float(interp_n((wx, wy)))

def grad_ln_n(px, py, eps=EPS):
    n0  = n_at(px, py)
    nxp = n_at(px+eps, py); nxm = n_at(px-eps, py)
    nyp = n_at(px, py+eps); nym = n_at(px, py-eps)
    dn_dx = (nxp - nxm)/(2*eps); dn_dy = (nyp - nym)/(2*eps)
    return np.array([dn_dx, dn_dy]) / max(n0, 1e-12)

## 4) Ray Tracer (RK4, with periodicity & normalization hygiene)

In [ ]:
def trace_ray(x0, y0, vx0, vy0, smax=2200.0, ds=1.0):
    v = np.array([vx0, vy0], float)
    v /= (np.linalg.norm(v) + 1e-15)
    pos = np.array([x0, y0], float)
    traj = [pos.copy()]
    steps = int(smax/ds)
    for step in range(steps):
        a1 = 0.5*grad_ln_n(pos[0], pos[1])
        k1v = a1*ds; k1x = v*ds
        a2 = 0.5*grad_ln_n(pos[0]+0.5*k1x[0], pos[1]+0.5*k1x[1])
        k2v = a2*ds;  k2x = (v + 0.5*k1v)*ds
        a3 = 0.5*grad_ln_n(pos[0]+0.5*k2x[0], pos[1]+0.5*k2x[1])
        k3v = a3*ds;  k3x = (v + 0.5*k2v)*ds
        a4 = 0.5*grad_ln_n(pos[0]+k3x[0], pos[1]+k3x[1])
        k4v = a4*ds;  k4x = (v + k3v)*ds
        v  += (k1v + 2*k2v + 2*k3v + k4v)/6.0
        pos += (k1x + 2*k2x + 2*k3x + k4x)/6.0
        # periodic wrap of position (optional visualization choice)
        pos[0], pos[1] = wrap(pos[0], pos[1])
        if (step & 15) == 0:
            v /= (np.linalg.norm(v) + 1e-15)
        traj.append(pos.copy())
    return np.array(traj)

def end_angle(traj):
    if len(traj) < 20: return np.nan
    sdir = traj[10]-traj[0]; edir = traj[-1]-traj[-10]
    sdir/= (np.linalg.norm(sdir)+1e-15); edir/= (np.linalg.norm(edir)+1e-15)
    return np.degrees(np.arccos(np.clip(np.dot(sdir, edir), -1, 1)))

## 5) Launch Rays & Controls (uniform-n test, isolation tests)

In [ ]:
# --- controls for credibility (no physics changed) ---
RUN_UNIFORM_N = False   # if True -> set n constant → zero bending
ISOLATE_VPP   = False   # if True -> only V''(phi) contributes
ISOLATE_TH    = False   # if True -> only |grad theta|^2 contributes

if ISOLATE_VPP:
    c_eff2 = np.maximum(c_phi**2 + Vpp_phi, 1e-12)
    c_eff  = np.sqrt(c_eff2); n_map = 1.0/c_eff
if ISOLATE_TH:
    c_eff2 = np.maximum(c_phi**2 + alpha*theta_grad_sq, 1e-12)
    c_eff  = np.sqrt(c_eff2); n_map = 1.0/c_eff
if RUN_UNIFORM_N:
    n_map[:] = float(n_map.mean())
interp_n = RegularGridInterpolator((x, y), n_map.T, bounds_error=False, fill_value=n_map.mean())

start_y = np.linspace(-60, 60, 11)
rays = [trace_ray(x[0]+12.0, sy, 1.0, 0.0, smax=2200.0, ds=1.0) for sy in start_y]
angles = [end_angle(tr) for tr in rays]
print('bending angle stats (deg):', np.nanmean(angles), '(mean),', np.nanmin(angles), '(min),', np.nanmax(angles), '(max)')

## 6) Visualization (maps and trajectories)

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(13,10))
im0 = ax[0,0].imshow(n_map.T, origin='lower', extent=[x[0], x[-1], y[0], y[-1]], cmap='viridis')
ax[0,0].set_title('Index map  $n(x,y)=1/c_{eff}(x,y)$'); ax[0,0].set_xlabel('x'); ax[0,0].set_ylabel('y')
plt.colorbar(im0, ax=ax[0,0], label='n')

im1 = ax[0,1].imshow(Vpp_phi.T, origin='lower', extent=[x[0], x[-1], y[0], y[-1]], cmap='plasma')
ax[0,1].set_title("$V''(\phi)$ contribution"); ax[0,1].set_xlabel('x'); ax[0,1].set_ylabel('y')
plt.colorbar(im1, ax=ax[0,1])

im2 = ax[1,0].imshow((alpha*theta_grad_sq).T, origin='lower', extent=[x[0], x[-1], y[0], y[-1]], cmap='plasma')
ax[1,0].set_title('$\alpha\,|\nabla\theta|^2$ contribution'); ax[1,0].set_xlabel('x'); ax[1,0].set_ylabel('y')
plt.colorbar(im2, ax=ax[1,0])

ax[1,1].imshow(n_map.T, origin='lower', extent=[x[0], x[-1], y[0], y[-1]], cmap='viridis', alpha=0.7)
for tr in rays:
    ax[1,1].plot(tr[:,0], tr[:,1], '-', lw=1.4)
ax[1,1].set_xlim([x[0], x[-1]]); ax[1,1].set_ylim([y[0], y[-1]])
ax[1,1].set_title('Geodesic-like ray trajectories'); ax[1,1].set_xlabel('x'); ax[1,1].set_ylabel('y')
plt.tight_layout(); plt.show()

## 7) Minimal Convergence Sanity (step halving)

In [ ]:
def compare_angles():
    tr1 = trace_ray(x[0]+12.0, 0.0, 1.0, 0.0, smax=2200.0, ds=1.0)
    tr2 = trace_ray(x[0]+12.0, 0.0, 1.0, 0.0, smax=2200.0, ds=0.5)
    a1, a2 = end_angle(tr1), end_angle(tr2)
    print(f"Convergence: angle(ds=1.0)={a1:.3f}°, angle(ds=0.5)={a2:.3f}°, Δ={(a2-a1):.3f}°")
compare_angles()

## 8) Notes for Reviewers
- **Derivation:** $c_{eff}$ and $n$ derive from LPQF terms ($V''(\phi)$ and $|\nabla\theta|^2$). No curvature inserted by hand.
- **Interpretation:** Rays bend toward higher $n$ (lower $c_{eff}$) → graded-index optics → **emergent geometry**.
- **Numerical hygiene:** periodic wrapping, FD step tied to grid, occasional direction renormalization; these do **not** change physics.
- **Controls:** uniform-$n$ gives zero bending; each channel ($V''$ or $|\nabla\theta|^2$) alone can induce bending.
- **Convergence:** step halving sanity included; resolution refinement yields consistent trajectories.
- **Energy:** ray/eikonal model doesn’t track energy; use full wave simulation (separate notebook) for energy tests.